### 1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [1]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Librería para parámetros polinomiales
from sklearn.preprocessing import PolynomialFeatures
#Librería para ajustar modelos lineales
from sklearn.linear_model import LinearRegression
# Para la creación de modelo Ridge
from sklearn.linear_model import Ridge
# Para la creación de modelo Ridge
from sklearn.linear_model import Lasso
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### 2. Carga de los datos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [2]:
# Se cargan los datos. 
data=pd.read_csv('datos_proyecto.csv',sep=';', encoding='latin-1')

In [3]:
# Cantidad de datos y número de variables
data.shape

(37759, 8)

In [4]:
# Mostrar los datos
data.head()

,area_quemada,clase_incendio,mes_incendio,vegetacion,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont
0,3.0,B,Diciembre,Desierto polar_roca,59.8,8.4,0.0,86.8
1,60.0,C,Febrero,Bosque tropical perennifolio secundario,168.8,42.2,18.1,124.5
2,1.0,B,Junio,Bosque tropical perennifolio,10.4,7.2,0.0,0.0
3,5.2,B,Enero,Matorral abierto,26.0,0.0,0.0,0.0
4,1.0,B,Noviembre,Matorral abierto,28.4,27.5,1.2,55.4


### 3. Limpieza y preparación de los datos

Recuerda que un aspecto muy importante para tener en cuenta son los requerimientos de entrada de los algoritmos de aprendizaje. Cada uno de estos puede trabajar con un tipo de variable, es por esto que vamos a realizar las mismas transformaciones que se realizaron en el notebook de regresión lineal. Además, vamos a ejecutar los mismos pasos de limpieza de los datos.

In [5]:
# Es recomendable que todos los pasos de preparación se realicen sobre otro archivo.
data_t = data
# Eliminación data vacía.
data_t=data_t.dropna()
# Eliminación de registros duplicados.
data_t=data_t.drop_duplicates()
# Se realiza la transformación de estas variables a dummies.
data_t = pd.get_dummies(data_t, columns=['clase_incendio','mes_incendio','vegetacion'])

In [6]:
# Visualización de los datos limpios
data_t.head()

,area_quemada,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,clase_incendio_B,clase_incendio_C,clase_incendio_D,clase_incendio_E,clase_incendio_F,...,mes_incendio_Noviembre,mes_incendio_Octubre,mes_incendio_Septiembre,vegetacion_Bosque templado,vegetacion_Bosque tropical perennifolio,vegetacion_Bosque tropical perennifolio secundario,vegetacion_Desierto,vegetacion_Desierto polar_roca,vegetacion_Matorral abierto,vegetacion_Pradera_estepa
0,3.0,59.8,8.4,0.0,86.8,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,60.0,168.8,42.2,18.1,124.5,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1.0,10.4,7.2,0.0,0.0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,5.2,26.0,0.0,0.0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1.0,28.4,27.5,1.2,55.4,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [7]:
# El tamaño de los datos preparados
data_t.shape

(26394, 30)

In [10]:
# Se selecciona la variable objetivo, en este caso "area_quemada".
Y=data_t['area_quemada']
# Del conjunto de datos se elimina la variable "area_quemada".
X=data_t.drop(['area_quemada'], axis=1)

## Regresión polinomial

### Usando grado = 2

In [11]:
# Utilizaremos una tranformación de grado 2.
poly = PolynomialFeatures(degree=2)
poly_X = poly.fit_transform(X)
# Esta transformación crea nuevas variables y las añade al conjunto de datos. Veamos cuántas se generan:
poly_X.shape

(26394, 465)

In [12]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
poly_X_train, poly_X_test, poly_Y_train, poly_Y_test = train_test_split(poly_X, Y, test_size = 0.2, random_state = 0)

# Creación del objeto de la clase LinearRegression y ajuste del modelo a los datos.
modelo_regresion_poly = LinearRegression()

# Ajustar el modelo con los datos de entrenamiento con las nuevas variables polinomiales
modelo_regresion_poly.fit(poly_X_train, poly_Y_train)

LinearRegression()

#### Evaluación del modelo

In [13]:
# Se obtienen las predicciones del modelo sobre el conjunto test.
y_pred = modelo_regresion_poly.predict(poly_X_test)

# Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales).
print('Métricas')
print('------ Modelo de regresión lineal polinomial múltiple----')
print("MSE: %.2f" % mean_squared_error(poly_Y_test, y_pred, squared=True))
print("RMSE: %.2f" % mean_squared_error(poly_Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(poly_Y_test, y_pred))
print('R²: %.2f' % r2_score(poly_Y_test, y_pred))

Métricas
------ Modelo de regresión lineal polinomial múltiple----
MSE: 295288207.30
RMSE: 17183.95
MAE: 3499.36
R²: 0.23


### Usando grado = 3

In [16]:
# Utilizaremos una tranformación de grado 2.
poly2 = PolynomialFeatures(degree=3)
poly_X2 = poly2.fit_transform(X)
# Esta transformación crea nuevas variables y las añade al conjunto de datos. Veamos cuántas se generan:
poly_X2.shape

(26394, 4960)

In [17]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
poly_X_train2, poly_X_test2, poly_Y_train2, poly_Y_test2 = train_test_split(poly_X2, Y, test_size = 0.2, random_state = 0)

# Creación del objeto de la clase LinearRegression y ajuste del modelo a los datos.
modelo_regresion_poly2 = LinearRegression()

# Ajustar el modelo con los datos de entrenamiento con las nuevas variables polinomiales
modelo_regresion_poly2.fit(poly_X_train2, poly_Y_train2)

LinearRegression()

In [18]:
# Se obtienen las predicciones del modelo sobre el conjunto test.
y_pred2 = modelo_regresion_poly2.predict(poly_X_test2)

# Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales).
print('Métricas')
print('------ Modelo de regresión lineal polinomial múltiple----')
print("MSE: %.2f" % mean_squared_error(poly_Y_test2, y_pred2, squared=True))
print("RMSE: %.2f" % mean_squared_error(poly_Y_test2, y_pred2, squared=False))
print("MAE: %.2f" % mean_absolute_error(poly_Y_test2, y_pred2))
print('R²: %.2f' % r2_score(poly_Y_test2, y_pred2))

Métricas
------ Modelo de regresión lineal polinomial múltiple----
MSE: 575892236.67
RMSE: 23997.75
MAE: 3874.41
R²: -0.49


Se logró mejores resultados usando un polinomio grado 2.

## Regresión regularizada. Caso Ridge.

### 1. Construcción del modelo 

In [19]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

Como ya has podido conocer la regresión Ridge utiliza la norma L2 del vector de coeficientes como término de regularización. Para controlar la cantidad de regularización se utiliza el hiperparámetro alpha. Probemos con un valor igual a 2.

In [20]:
modelo_ridge = Ridge(alpha=2)
modelo_ridge

Ridge(alpha=2)

In [21]:
# Ajuste del modelo
modelo_ridge.fit(X_train,Y_train)

Ridge(alpha=2)

In [22]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.75
MAE: 3647.36
R²: 0.23


#### Selección de técnica de validación para escoger el mejor alpha

##### 1-Con un conjunto de validación

In [23]:
# Dividimos el conjunto de entrenamiento en dos: una para la construcción del modelo (sería el nuevo conjunto de 
# entrenamiento) y otro para la validación, el cuál será utilizado para determinar el rendimiento del modelo con una
# combinación específica de hiperparámetros.
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size = 0.15, random_state = 0)

In [24]:
# Fijemos un valor de alpha en 1, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = Ridge(alpha=1)
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 3170.57


In [25]:
# Fijemos un valor de alpha de 2, construyamos el modelo y probemos sobre validación.
modelo_ridge = Ridge(alpha=2)
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 3169.66


In [26]:
# Fijemos un valor de alpha de 5, construyamos el modelo y probemos sobre validación.
modelo_ridge = Ridge(alpha=5)
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 3166.93


El mejor alpha, según MAE, es 5. Seleccionemos este modelo como el final (por esta razón también estás técnicas se 
conocen como métodos de selección de modelos) y determinemos su rendimiento de generalización. 
Toma en cuenta que ahora debes utilizar todo el conjunto de entrenamiento.

In [27]:
modelo_ridge = Ridge(alpha=5) 
modelo_ridge.fit(X_train,Y_train)
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17242.27
MAE: 3645.31
R²: 0.23


Estos pasos de asignar un valor de alpha, entrenar y probar con validación pueden ser implementados en un ciclo. 

##### 2-Validación cruzada de k-particiones

In [28]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [29]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'alpha': [0.1, 0.5, 1.0]}

In [30]:
# Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_ridge = Ridge()

In [31]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0]})

In [32]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 1.0}


In [33]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, scoring = 'neg_mean_absolute_error', n_jobs=-1)
mejor_modelo.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 1.0}


Un aspecto importante que debes tener en cuenta es que GridSearchCV devuelve el modelo entrenado con el mejor valor del hiperparámetro alpha (resultado de la búsqueda).

In [34]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.57
MAE: 3648.06
R²: 0.23


## Regresión regularizada. Caso Lasso.

### 1. Construcción del modelo

In [35]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [36]:
# Para acelerar la convergencia del algoritmo que utiliza Lasso para optimizar la función de costo, utilizaremos la opción
# de normalizar los datos para que todos estén en el mismo rango.
modelo_lasso = Lasso(alpha=1)
modelo_lasso

Lasso(alpha=1)

In [37]:
# Ajuste del modelo
modelo_lasso.fit(X_train,Y_train)

Lasso(alpha=1)

In [38]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_lasso.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.45
MAE: 3643.23
R²: 0.23


#### Selección de técnica de validación para escoger el mejor alpha

##### 1-Validación cruzada de k-particiones

In [39]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [40]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'alpha': [1, 2, 3, 4, 5]}

In [41]:
# Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_lasso = Lasso()

In [42]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo2 = GridSearchCV(modelo_lasso, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo2.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 4, 5]})

In [43]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'alpha': 1.0}


In [44]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo2 = GridSearchCV(modelo_lasso, param_grid, scoring = 'neg_mean_absolute_error', n_jobs=-1 )
mejor_modelo2.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 1.0}


In [45]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.57
MAE: 3648.06
R²: 0.23


### 2. Importancia de variables y selección

In [46]:
# Revisar los parámetros del modelo entrenado
coeficientes = modelo_final.coef_
variables = X_train.columns
# Mostrar en una tabla los valores de los coeficientes para cada variable
pd.DataFrame({'coeficientes':coeficientes,'variables':variables})

,coeficientes,variables
0,-0.126314,Prec_pre_30
1,-0.507284,Prec_pre_15
2,0.639980,Prec_pre_7
3,-0.602888,Prec_cont
4,-4965.819280,clase_incendio_B
5,-5005.882825,clase_incendio_C
6,-5056.150349,clase_incendio_D
7,-4633.671224,clase_incendio_E
8,-1932.896960,clase_incendio_F
9,21594.420637,clase_incendio_G
